In [54]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [55]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_thunder_3.tflite")
interpreter.allocate_tensors()

In [56]:
lowerjoints_list = [13,14,15,16]

In [57]:
def locating_joints(frame, confidence_threshold):
    #y, x, c = frame.shape
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [58]:
def knee_distance(frame):
   # y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [59]:
def left_slope(frame):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [60]:
def ankle_distance(frame):
    #y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [61]:
def right_slope(frame):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [62]:
EDGES = {
    (13, 15): 'm',
    (14, 16): 'm',
    (13, 14): 'm',
    (16,15) : 'm'
}

In [63]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    #y, x, c = frame.shape
    print(keypoints.shape)

    for edge, color in edges.items():
        e1, e2 = edge
        y1, x1, c1 = joints_coordinates[e1]
        y2, x2, c2 = joints_coordinates[e2]
        
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [64]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  right_m= (y4-y2)/(x4-x2)
  left_m = (y3-y1)/(x3-x1)
  print("knee:", dis)
  print("ankle:", ankle_dis)
  print("left",left_m)
  print("Right",right_m)
#8739.635093408378
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  ankle = (x4-x3)**2+(y4-y3)**2
  #dis below 88 and ankle below 88, dis >80 and ankle > 80
  if (dis <= 7195.855191894665 and dis > 5608.360385571377 and ankle_dis <= 7195.855191894665 and ankle_dis > 5608.855051436771) or (dis < 7195.855191894665 and dis > 6495.855191894665 and ankle_dis >= 7195.855191894665 and ankle_dis <= 7795.855191894665 ):
     print("running loop 1")
     cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 5608.855051436771 and ankle_dis <= 5608.855051436771) or (dis < 5608.855051436771 and ankle_dis > 5608.855051436771 and ankle_dis <= 5908.855051436771):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis >= 7195.855191894665 and ankle_dis >= 7195.855191894665):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
     cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     pass
            
       
        
        

        
    
    

In [65]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0),256,256)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    # predicting joints
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("hello boy")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("gurs")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("Kaur")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    locating_joints(frame, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame)
    ankle_distance(frame)
    left_slope(frame)
    right_slope(frame)
    insoles(frame)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
hello boy
[[[[0.5756109  0.53726226 0.68968636]
   [0.50477403 0.58667594 0.8450297 ]
   [0.5127318  0.47390965 0.83649576]
   [0.511875   0.641558   0.70565426]
   [0.5194645  0.40338904 0.827628  ]
   [0.80062395 0.83589625 0.8948165 ]
   [0.7784337  0.26306015 0.79380655]
   [0.8674835  0.933113   0.01251698]
   [0.8391178  0.31498924 0.00380561]
   [0.5775916  0.6129858  0.05482906]
   [0.8345238  0.36860487 0.01099986]
   [0.9084116  0.87066805 0.00203127]
   [0.42385736 0.55982745 0.00767055]
   [0.56763613 0.8898588  0.00860909]
   [0.7913179  0.2639731  0.00552788]
   [0.4985973  0.5754664  0.01881933]
   [0.8165399  0.46088055 0.00751

In [66]:
cap.release()
cv2.destroyAllWindows()